In [22]:
path_to_src = '/content/drive/MyDrive/Colab Notebooks/GNN/MoTSE-main/src'
"""dataset_name = "QM9"
task_lst = ["mu","alpha","homo","lumo","gap","r2","zpve","u0","u298","h298","g298","cv"]"""
dataset_name = "PCBA"
task_lst = ["PCBA-1030","PCBA-1458","PCBA-1460","PCBA-2546","PCBA-2551","PCBA-485297","PCBA-485313","PCBA-485364","PCBA-504332","PCBA-504333","PCBA-504339","PCBA-504444","PCBA-504467","PCBA-588342","PCBA-624296","PCBA-624297","PCBA-624417","PCBA-651965","PCBA-652104","PCBA-686970","PCBA-686978","PCBA-686979","PCBA-720504"]

#If using Google drive
from google.colab import drive
drive.mount('/content/drive')
# get the current working directory
import sys
sys.path.insert(0,path_to_src)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!echo $CUDA_VERSION

11.8.0


In [18]:
#!pip3 uninstall dgl -y
!pip3 install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html
!pip3 install dgllife
!pip3 install rdkit

Looking in links: https://data.dgl.ai/wheels/cu118/repo.html


In [32]:
import torch
import pandas as pd
from utils import load_model, makedir, set_random_seed
from utils.data import load_data
from trainer import Trainer

%load_ext autoreload
%autoreload 2
set_random_seed(22) #Test 18 next

task_lst = task_lst[:10] #Only testing transfer to first 10 tasks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
tasks = task_lst
data_path = path_to_src + f'/../datasets/{dataset_name}/1000/'
model_type = 'GCN'
model_path = path_to_src + f"/../saved_models/{dataset_name}/GCN/1000/"
makedir(model_path)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
results_dict = {'task':[]}

In [34]:
for task in tasks:
    print(task)
    train_loader, val_loader, test_loader, data_args = load_data(
        dataset=dataset_name,
        data_path=data_path,
        tasks=[task],
        device = device
    )
    model = load_model(device=device)
    trainer = Trainer(device=device,tasks=[task],
                      data_args=data_args,model_path=model_path,
                     )
    model, task_results_dict = trainer.fit(model, train_loader,
                                      val_loader, test_loader)
    results_dict['task'].append(task)
    for metric in data_args['metrics']:
        if metric not in list(results_dict.keys()):
            results_dict.update({metric:[]})
        results_dict[metric].append(task_results_dict[metric][task])

result_path = model_path.replace('saved_models','results')
makedir(result_path)
pd.DataFrame(results_dict).to_csv(result_path+'results.csv', float_format='%.3f',
                                  index=False)
print(f"Results have been saved to {result_path+'results.csv'}")

PCBA-1030
819 loaded!
100 loaded!
980 loaded!
[0] training loss:0.6924574106347328
val roc_auc:0.5794317727090836
val auprc:0.6167564888732405
[20] training loss:0.6471084169313019
val roc_auc:0.6606642657062824
val auprc:0.6625551419038418
[40] training loss:0.6344236375070086
val roc_auc:0.6934773909563825
val auprc:0.6966737344693117
[60] training loss:0.6210583068576514
val roc_auc:0.7034813925570228
val auprc:0.6981438943210407
[80] training loss:0.6059615600342844
val roc_auc:0.7006802721088435
val auprc:0.6946377338910682
test roc_auc:0.6126127251900448
test auprc:0.5823192151122943
PCBA-1458
824 loaded!
92 loaded!
976 loaded!
[0] training loss:0.6938596367835999
val roc_auc:0.7736294896030246
val auprc:0.746038621999146
[20] training loss:0.6134197039931428
val roc_auc:0.7972589792060492
val auprc:0.746488706231806
[40] training loss:0.5913142819030612
val roc_auc:0.8275047258979206
val auprc:0.7892829971204361
[60] training loss:0.604651939050824
val roc_auc:0.8397920604914934